<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%%time
%pylab inline
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import gc
gc.enable()
import sys

from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.cluster import *
from sklearn.pipeline import *
from sklearn.preprocessing import *
import lightgbm as lgb
import catboost as ctbst
from boruta import BorutaPy

import tensorflow
KERAS_BACKEND=tensorflow
import keras
from keras import *
from keras.utils import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.layers import *
from keras.layers.core import *
from keras.layers.recurrent import *
from keras.layers.normalization import *
from keras.layers.advanced_activations import *
from keras import backend as K
from keras.utils.vis_utils import model_to_dot

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 10
plt.style.use('fivethirtyeight')
import seaborn as sns
import plotly
from plotly.offline import *
import plotly.plotly as py
from plotly.graph_objs import *
from pylab import *
init_notebook_mode(connected=True)
import shap
from tqdm import *
from IPython.display import *

import threading as th
import multiprocessing as mp
import itertools

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


Wall time: 15.3 s


In [2]:
#cannot use seed cause python has function with the same name
random_state = 1
path = 'D:/Python/Kaggle/VSB/'
n_threads = mp.cpu_count()-1 #th.active_count()

# select how many folds will be created
N_SPLITS = 10
# it is just a constant with the measurements data size
sample_size = 800000

In [3]:
# It is the official metric used in this competition
# below is the declaration of a function used inside the keras model, calculation with K (keras backend / thensorflow)
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [4]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [5]:
# just load train data
df_train = pd.read_csv(path + 'metadata_train.csv')
# set index, it makes the data access much faster
df_train = df_train.set_index(['id_measurement', 'phase'])
df_train.head()

signal_id  target
id_measurement phase                   
0              0              0       0
               1              1       0
               2              2       0
1              0              3       1
               1              4       1

In [6]:
# in other notebook I have extracted the min and max values from the train data, the measurements
max_num = 127
min_num = -128

In [7]:
# This function standardize the data from (-128 to 127) to (-1 to 1)
# Theoretically it helps in the NN Model training, but I didn't tested without it
def min_max_transf(ts, min_data, max_data, range_needed=(-1,1)):
    if min_data < 0:
        ts_std = (ts + abs(min_data)) / (max_data + abs(min_data))
    else:
        ts_std = (ts - min_data) / (max_data - min_data)
    if range_needed[0] < 0:    
        return ts_std * (range_needed[1] + abs(range_needed[0])) + range_needed[0]
    else:
        return ts_std * (range_needed[1] - range_needed[0]) + range_needed[0]

In [8]:
# This is one of the most important peace of code of this Kernel
# Any power line contain 3 phases of 800000 measurements, or 2.4 millions data 
# It would be praticaly impossible to build a NN with an input of that size
# The ideia here is to reduce it each phase to a matrix of <n_dim> bins by n features
# Each bin is a set of 5000 measurements (800000 / 160), so the features are extracted from this 5000 chunk data.
def transform_ts(ts, n_dim=160, min_max=(-1,1)):
    # convert data into -1 to 1
    ts_std = min_max_transf(ts, min_data=min_num, max_data=max_num)
    # bucket or chunk size, 5000 in this case (800000 / 160)
    bucket_size = int(sample_size / n_dim)
    # new_ts will be the container of the new data
    new_ts = []
    # this for iteract any chunk/bucket until reach the whole sample_size (800000)
    for i in range(0, sample_size, bucket_size):
        # cut each bucket to ts_range
        ts_range = ts_std[i:i + bucket_size]
        # calculate each feature
        mean = ts_range.mean()
        std = ts_range.std() # standard deviation
        std_top = mean + std # I have to test it more, but is is like a band
        std_bot = mean - std
        #sm = ts_range.sum()
        #mn = ts_range.min()
        #mx = ts_range.max()
        #md = ts_range.median()
        
        # I think that the percentiles are very important, it is like a distribuiton analysis from each chunk
        percentil_calc = np.percentile(ts_range, [0, 1, 25, 50, 75, 99, 100])
        max_range = percentil_calc[-1] - percentil_calc[0] # this is the amplitude of the chunk
        relative_percentile = percentil_calc - mean # maybe it could heap to understand the asymmetry
        # now, we just add all the features to new_ts and convert it to np.array
        new_ts.append(np.concatenate([np.asarray([mean, std, std_top, std_bot, max_range]), percentil_calc, relative_percentile]))
    return np.asarray(new_ts)

In [9]:
# this function take a piece of data and convert using transform_ts(), but it does to each of the 3 phases
# if we would try to do in one time, could exceed the RAM Memmory
def prep_data(start, end):
    # load a piece of data from file
    praq_train = pq.read_pandas(path + 'train.parquet', columns=[str(i) for i in range(start, end)]).to_pandas()
    X = []
    y = []
    # using tdqm to evaluate processing time
    # takes each index from df_train and iteract it from start to end
    # it is divided by 3 because for each id_measurement there are 3 id_signal, and the start/end parameters are id_signal
    for id_measurement in tqdm_notebook(df_train.index.levels[0].unique()[int(start/3):int(end/3)]):
        X_signal = []
        # for each phase of the signal
        for phase in [0,1,2]:
            # extract from df_train both signal_id and target to compose the new data sets
            signal_id, target = df_train.loc[id_measurement].loc[phase]
            # but just append the target one time, to not triplicate it
            if phase == 0:
                y.append(target)
            # extract and transform data into sets of features
            X_signal.append(transform_ts(praq_train[str(signal_id)]))
        # concatenate all the 3 phases in one matrix
        X_signal = np.concatenate(X_signal, axis=1)
        # add the data to X
        X.append(X_signal)
    X = np.asarray(X)
    y = np.asarray(y)
    return X, y

In [10]:
# this code is very simple, divide the total size of the df_train into two sets and process it
X = []
y = []
def load_all():
    total_size = len(df_train)
    for ini, end in [(0, int(total_size/2)), (int(total_size/2), total_size)]:
        X_temp, y_temp = prep_data(ini, end)
        X.append(X_temp)
        y.append(y_temp)
load_all()

#X, y = prep_data(0, len(df_train))

X = np.concatenate(X)
y = np.concatenate(y)

OSError: Passed non-file path: /Python/Kaggle/VSB/train.parquet

In [17]:
path

'D:/Python/Kaggle/VSB/'

In [43]:
# The X shape here is very important. It is also important undertand a little how a LSTM works
# X.shape[0] is the number of id_measuremts contained in train data
# X.shape[1] is the number of chunks resultant of the transformation, each of this date enters in the LSTM serialized
# This way the LSTM can understand the position of a data relative with other and activate a signal that needs
# a serie of inputs in a specifc order.
# X.shape[3] is the number of features multiplied by the number of phases (3)
print(X.shape, y.shape)
# save data into file, a numpy specific format
np.save("X.npy",X)
np.save("y.npy",y)

(2904, 160, 57) (2904,)


In [44]:
# This is NN LSTM Model creation
def model_lstm(input_shape):
    # The shape was explained above, must have this order
    inp = Input(shape=(input_shape[1], input_shape[2],))
    # This is the LSTM layer
    # Bidirecional implies that the 160 chunks are calculated in both ways, 0 to 159 and 159 to zero
    # although it appear that just 0 to 159 way matter, I have tested with and without, and tha later worked best
    # 128 and 64 are the number of cells used, too many can overfit and too few can underfit
    x = Bidirectional(CuDNNLSTM(512, return_sequences=True))(inp)
    x = Bidirectional(CuDNNLSTM(256, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    # The second LSTM can give more fire power to the model, but can overfit it too
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    # Attention is a new tecnology that can be applyed to a Recurrent NN to give more meanings to a signal found in the middle
    # of the data, it helps more in longs chains of data. A normal RNN give all the responsibility of detect the signal
    # to the last cell. Google RNN Attention for more information :)
    x = Attention(input_shape[1])(x)
    # A intermediate full connected (Dense) can help to deal with nonlinears outputs
    x = Dense(64, activation="relu")(x)
    # A binnary classification as this must finish with shape (1,)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    # Pay attention in the addition of matthews_correlation metric in the compilation, it is a success factor key
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[matthews_correlation])
    
    return model

In [45]:
%%time

# Here is where the training happens

# First, create a set of indexes of the 5 folds
splits = list(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=random_state).split(X, y))
preds_val = []
y_val = []
# Then, iteract with each fold
# If you dont know, enumerate(['a', 'b', 'c']) returns [(0, 'a'), (1, 'b'), (2, 'c')]
for idx, (train_idx, val_idx) in enumerate(splits):
    K.clear_session() # I dont know what it do, but I imagine that it "clear session" :)
    print("Beginning fold {}".format(idx+1))
    # use the indexes to extract the folds in the train and validation data
    train_X, train_y, val_X, val_y = X[train_idx], y[train_idx], X[val_idx], y[val_idx]
    # instantiate the model for this fold
    model = model_lstm(train_X.shape)
    # This checkpoint helps to avoid overfitting. It just save the weights of the model if it delivered an
    # validation matthews_correlation greater than the last one.
    ckpt = ModelCheckpoint('weights_{}.h5'.format(idx), save_best_only=True, save_weights_only=True, verbose=1, monitor='val_matthews_correlation', mode='max')
    # Train, train, train
    model.fit(train_X, train_y, batch_size=128, epochs=50, validation_data=[val_X, val_y], callbacks=[ckpt])
    # loads the best weights saved by the checkpoint
    model.load_weights('weights_{}.h5'.format(idx))
    # Add the predictions of the validation to the list preds_val
    preds_val.append(model.predict(val_X, batch_size=512))
    # and the val true y
    y_val.append(val_y)

# concatenates all and prints the shape    
preds_val = np.concatenate(preds_val)[...,0]
y_val = np.concatenate(y_val)
preds_val.shape, y_val.shape

Beginning fold 1
Train on 2613 samples, validate on 291 samples
Epoch 1/50
2613/2613 [==============================] - ETA: 47s - loss: 0.6943 - matthews_correlation: 0.092 - ETA: 25s - loss: 0.6203 - matthews_correlation: 0.046 - ETA: 18s - loss: 0.5204 - matthews_correlation: 0.030 - ETA: 14s - loss: 0.4632 - matthews_correlation: 0.023 - ETA: 12s - loss: 0.4255 - matthews_correlation: 0.018 - ETA: 10s - loss: 0.3987 - matthews_correlation: 0.015 - ETA: 8s - loss: 0.3809 - matthews_correlation: 0.013 - ETA: 7s - loss: 0.3652 - matthews_correlation: 0.01 - ETA: 6s - loss: 0.3432 - matthews_correlation: 0.01 - ETA: 6s - loss: 0.3303 - matthews_correlation: 0.00 - ETA: 5s - loss: 0.3242 - matthews_correlation: 0.00 - ETA: 4s - loss: 0.3091 - matthews_correlation: 0.00 - ETA: 3s - loss: 0.3070 - matthews_correlation: 0.00 - ETA: 3s - loss: 0.3002 - matthews_correlation: 0.00 - ETA: 2s - loss: 0.2890 - matthews_correlation: 0.00 - ETA: 2s - loss: 0.2896 - matthews_correlation: 0.00 - ETA

2613/2613 [==============================] - ETA: 7s - loss: 0.1675 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.2334 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.2689 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.2500 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2430 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2547 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2466 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2407 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2381 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2340 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2268 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2222 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2281 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2286 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2229 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2252 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2245 - ma

2613/2613 [==============================] - ETA: 7s - loss: 0.3760 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.3223 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.3068 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.2941 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2639 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2359 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2257 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2181 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2235 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2202 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2175 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2194 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2133 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2167 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2148 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2202 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2236 - ma

2613/2613 [==============================] - ETA: 6s - loss: 0.1527 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.2080 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.2184 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2152 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2209 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2114 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2099 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2145 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2154 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2196 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2198 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2203 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2236 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2212 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2242 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2272 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2281 - ma


Epoch 00041: val_matthews_correlation improved from 0.26392 to 0.49887, saving model to weights_0.h5
Epoch 42/50
2613/2613 [==============================] - ETA: 6s - loss: 0.1330 - matthews_correlation: 0.52 - ETA: 6s - loss: 0.1577 - matthews_correlation: 0.32 - ETA: 6s - loss: 0.1790 - matthews_correlation: 0.38 - ETA: 5s - loss: 0.1909 - matthews_correlation: 0.35 - ETA: 5s - loss: 0.1934 - matthews_correlation: 0.38 - ETA: 5s - loss: 0.1884 - matthews_correlation: 0.32 - ETA: 4s - loss: 0.1957 - matthews_correlation: 0.27 - ETA: 4s - loss: 0.1869 - matthews_correlation: 0.27 - ETA: 4s - loss: 0.1775 - matthews_correlation: 0.28 - ETA: 3s - loss: 0.1812 - matthews_correlation: 0.29 - ETA: 3s - loss: 0.1839 - matthews_correlation: 0.26 - ETA: 3s - loss: 0.1848 - matthews_correlation: 0.26 - ETA: 2s - loss: 0.1867 - matthews_correlation: 0.27 - ETA: 2s - loss: 0.1811 - matthews_correlation: 0.29 - ETA: 1s - loss: 0.1779 - matthews_correlation: 0.31 - ETA: 1s - loss: 0.1802 - matthe

...

2613/2613 [==============================] - ETA: 6s - loss: 0.1898 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.1786 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.1823 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.1898 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2124 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2237 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2284 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2347 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2322 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2366 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2382 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2311 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2313 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2315 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2263 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2319 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2308 - ma

2613/2613 [==============================] - ETA: 6s - loss: 0.2128 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.2128 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.2127 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2127 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2212 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2269 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2309 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2286 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2292 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2296 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2261 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2250 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2323 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2293 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2254 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2353 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2289 - ma

2613/2613 [==============================] - ETA: 6s - loss: 0.1305 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.1822 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.1858 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.1824 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2010 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2065 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2104 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2290 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2272 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2320 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2209 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2237 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2293 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2282 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2285 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2367 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2353 - ma

2613/2613 [==============================] - ETA: 6s - loss: 0.2130 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.2134 - matthews_correlation: 0.0000e+ - ETA: 6s - loss: 0.2071 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2190 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.2025 - matthews_correlation: 0.0000e+ - ETA: 5s - loss: 0.1948 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2033 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2021 - matthews_correlation: 0.0000e+ - ETA: 4s - loss: 0.2148 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2167 - matthews_correlation: 0.0000e+ - ETA: 3s - loss: 0.2239 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2212 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2206 - matthews_correlation: 0.0000e+ - ETA: 2s - loss: 0.2170 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2223 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2190 - matthews_correlation: 0.0000e+ - ETA: 1s - loss: 0.2237 - ma

KeyboardInterrupt: 

In [ ]:
#160 bins
#default model
Epoch 00050: val_matthews_correlation did not improve from 0.64946
Wall time: 13min 28s

In [ ]:
#160 bins
#with one more recurrent LSTM layer
Epoch 00050: val_matthews_correlation did not improve from 0.69118
Wall time: 26min 47s

In [38]:
# The output of this kernel must be binary (0 or 1), but the output of the NN Model is float (0 to 1).
# So, find the best threshold to convert float to binary is crucial to the result
# this piece of code is a function that evaluates all the possible thresholds from 0 to 1 by 0.01
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm_notebook([i * 0.01 for i in range(100)]):
        score = K.eval(
            matthews_correlation(
                tf.convert_to_tensor(y_true, np.float64),
                tf.convert_to_tensor((y_proba > threshold), np.float64))
        )
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'matthews_correlation': best_score}
    return search_result

In [39]:
best_threshold = threshold_search(y_val, preds_val)['threshold']

KeyboardInterrupt: 

In [ ]:
%%time
# Now load the test data
# This first part is the meta data, not the main data, the measurements
meta_test = pd.read_csv(path + 'metadata_test.csv')

In [ ]:
meta_test = meta_test.set_index(['signal_id'])
meta_test.head()

In [ ]:
%%time

# First we daclarete a series of parameters to initiate the loading of the main data
# it is too large, it is impossible to load in one time, so we are doing it in dividing in 10 parts
first_sig = meta_test.index[0]
n_parts = 10
max_line = len(meta_test)
part_size = int(max_line / n_parts)
last_part = max_line % n_parts
print(first_sig, n_parts, max_line, part_size, last_part, n_parts * part_size + last_part)
# Here we create a list of lists with start index and end index for each of the 10 parts and one for the last partial part
start_end = [[x, x+part_size] for x in range(first_sig, max_line + first_sig, part_size)]
start_end = start_end[:-1] + [[start_end[-1][0], start_end[-1][0] + last_part]]
print(start_end)
X_test = []
# now, very like we did above with the train data, we convert the test data part by part
# transforming the 3 phases 800000 measurement in matrix (160,57)
for start, end in start_end:
    subset_test = pq.read_pandas(path + 'test.parquet', columns=[str(i) for i in range(start, end)]).to_pandas()
    for i in tqdm(subset_test.columns):
        id_measurement, phase = meta_test.loc[int(i)]
        subset_test_col = subset_test[i]
        subset_trans = transform_ts(subset_test_col)
        X_test.append([i, id_measurement, phase, subset_trans])

In [ ]:
X_test_input = np.asarray([np.concatenate([X_test[i][3],X_test[i+1][3], X_test[i+2][3]], axis=1) for i in range(0,len(X_test), 3)])
np.save("X_test.npy",X_test_input)
X_test_input.shape

In [ ]:
submission = pd.read_csv(path + 'sample_submission.csv')
print(len(submission))
submission.head()

In [ ]:
preds_test = []
for i in range(N_SPLITS):
    model.load_weights('weights_{}.h5'.format(i))
    pred = model.predict(X_test_input, batch_size=300, verbose=1)
    pred_3 = []
    for pred_scalar in pred:
        for i in range(3):
            pred_3.append(pred_scalar)
    preds_test.append(pred_3)

In [ ]:
preds_test = (np.squeeze(np.mean(preds_test, axis=0)) > best_threshold).astype(np.int)
preds_test.shape

In [ ]:
submission['target'] = preds_test
submission.to_csv(path + 'submission.csv', index=False)
submission.head()